In [25]:
import numpy as np 
import pandas as pd
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler

In [26]:
df = pd.read_csv("./data/stress_class.csv")
df.head()

,Study_Hours_Per_Day,Sleep_Hours_Per_Day,GPA,Stress_Level
0,19,37,65,1
1,3,30,41,2
2,1,42,33,2
3,15,22,54,1
4,31,15,117,0


In [27]:
train , test = np.split(df.sample(frac=1), [int(0.7*len(df))])

c:\Users\nkca1\anaconda3\envs\trailio_dev\Lib\site-packages\numpy\_core\fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [28]:
train.head()

,Study_Hours_Per_Day,Sleep_Hours_Per_Day,GPA,Stress_Level
448,36,45,72,0
1347,2,8,24,0
201,47,50,133,0
531,48,48,125,0
1351,15,5,94,0


In [29]:
test.head()

,Study_Hours_Per_Day,Sleep_Hours_Per_Day,GPA,Stress_Level
1186,20,39,63,1
897,18,17,69,1
263,39,48,123,0
1242,42,25,114,0
1224,19,38,69,1


In [30]:
def scale_dataset(dataframe, oversample=False):
  X = dataframe[dataframe.columns[:-1]].values
  y = dataframe[dataframe.columns[-1]].values

  scaler = StandardScaler()
  X = scaler.fit_transform(X)

  if oversample:
    ros = RandomOverSampler()
    X, y = ros.fit_resample(X, y)

  return X, y

In [31]:
X_train, y_train = scale_dataset(train, oversample=True)
X_test, y_test = scale_dataset(test)

In [32]:
from sklearn.metrics import classification_report

In [34]:
from sklearn.linear_model import  LogisticRegression
log_reg = LogisticRegression()

log_reg.fit(X_train, y_train)

LogisticRegression()

In [38]:
y_pred = log_reg.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.87      0.80      0.83       315
           1       0.73      0.78      0.75       196
           2       0.83      0.92      0.87        89

    accuracy                           0.81       600
   macro avg       0.81      0.84      0.82       600
weighted avg       0.82      0.81      0.81       600



In [40]:
from sklearn.naive_bayes import  GaussianNB
gauss_nb = GaussianNB()

gauss_nb.fit(X_train, y_train)

GaussianNB()

In [42]:
y_pred = gauss_nb.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.92      0.80      0.86       315
           1       0.81      0.85      0.83       196
           2       0.72      0.97      0.82        89

    accuracy                           0.84       600
   macro avg       0.81      0.87      0.83       600
weighted avg       0.85      0.84      0.84       600



In [43]:
from sklearn.svm import SVC
svm = SVC()

svm.fit(X_train, y_train)

SVC()

In [44]:
y_pred = svm.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.99      0.97      0.98       315
           1       0.96      0.94      0.95       196
           2       0.90      0.98      0.94        89

    accuracy                           0.96       600
   macro avg       0.95      0.96      0.96       600
weighted avg       0.96      0.96      0.96       600

